## 預測價格

In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# --------------------------
# 1. 讀取資料
# --------------------------
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(file_path)

eco_raw = data[data['艙等（主航段）'] == '經濟艙'].copy()
biz_raw = data[data['艙等（主航段）'] == '商務艙'].copy()

# --------------------------
# 2. 欄位定義
# --------------------------
cat_cols = [
    '出發時段','出發機場代號','抵達時段','航空聯盟組合','航空公司組合',
    '航空聯盟','停靠站數量','機型分類','假期','飛行時間兩段分類','是否為平日'
]
num_cols = ['停留時間_分鐘','實際飛行時間_分鐘','經濟指標','機場指標','competing_flights']
log_col  = '平均價格_log'
true_col = '平均價格'

# --------------------------
# 3. 編碼與標準化
# --------------------------
def encode_and_scale(df_raw):
    cols = cat_cols + num_cols + [log_col, true_col]
    df_sub = df_raw[cols]
    df_enc = pd.get_dummies(df_sub, columns=cat_cols, drop_first=True)
    scaler = StandardScaler()
    df_enc[num_cols] = scaler.fit_transform(df_enc[num_cols])
    return df_enc, scaler

eco_enc, eco_scaler = encode_and_scale(eco_raw)
biz_enc, biz_scaler = encode_and_scale(biz_raw)

# 取共同欄位（移除 log, true）
common_feats = list(set(eco_enc.columns) & set(biz_enc.columns))
for c in [log_col, true_col]:
    if c in common_feats:
        common_feats.remove(c)

# --------------------------
# 4. 拆分訓練集 & 訓練模型
# --------------------------
def split_and_train(df_enc, label_col, seed=123):
    X = df_enc[common_feats]
    y = df_enc[label_col]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed
    )
    model = XGBRegressor(n_estimators=100, random_state=seed, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"🔹測試集 MSE={mean_squared_error(y_test, y_pred):.4f}, "
          f"R2={r2_score(y_test, y_pred):.4f}")
    sigma2 = np.mean((y_test - y_pred)**2)
    return model, sigma2, df_enc.loc[X_test.index]

eco_model, sigma2_eco, eco_test = split_and_train(eco_enc, log_col)
biz_model, sigma2_biz, biz_test = split_and_train(biz_enc, log_col)

# --------------------------
# 5. 測試集資料：擷取組合
# --------------------------
def get_real_combinations(df_raw, code, cabin):
    sub = df_raw[(df_raw['抵達機場代號'] == code) & (df_raw['艙等（主航段）'] == cabin)]
    cols = cat_cols + num_cols + [true_col]
    return sub[cols].drop_duplicates().reset_index(drop=True)

# --------------------------
# 6. 測試集預測與儲存
# --------------------------
def predict_and_save_testset(test_df, model, scaler, sigma2, cabin, prefix):
    out_dir = 'predict_data/long'
    os.makedirs(out_dir, exist_ok=True)
    airports = sorted(test_df['抵達機場代號'].unique())

    for code in airports:
        combos = get_real_combinations(test_df, code, cabin)
        if combos.empty:
            continue
        sub = combos[cat_cols + num_cols]
        enc = pd.get_dummies(sub, columns=cat_cols, drop_first=True)
        enc = enc.reindex(columns=common_feats, fill_value=0)
        enc[num_cols] = scaler.transform(combos[num_cols])
        log_preds = model.predict(enc)
        combos['預測_平均價格'] = np.expm1(log_preds + 0.5 * sigma2)
        combos['抵達機場代號'] = code
        combos = combos.drop_duplicates(subset=cat_cols + num_cols, keep='last')
        fn = os.path.join(out_dir, f"{prefix}_{code}.csv")
        combos.to_csv(fn, index=False)
        print(f"✅ 測試集已存：{fn}（共 {len(combos)} 筆）")

# --------------------------
# 7. 執行測試集預測
# --------------------------
predict_and_save_testset(eco_raw.loc[eco_test.index], eco_model, eco_scaler, sigma2_eco, '經濟艙', 'eco')
predict_and_save_testset(biz_raw.loc[biz_test.index], biz_model, biz_scaler, sigma2_biz, '商務艙', 'biz')

🔹測試集 MSE=0.0319, R2=0.8960
🔹測試集 MSE=0.0165, R2=0.8761
✅ 測試集已存：predict_data/long/eco_CDG.csv（共 1576 筆）
✅ 測試集已存：predict_data/long/eco_FRA.csv（共 2120 筆）
✅ 測試集已存：predict_data/long/eco_JFK.csv（共 1221 筆）
✅ 測試集已存：predict_data/long/eco_LAX.csv（共 2293 筆）
✅ 測試集已存：predict_data/long/eco_LHR.csv（共 2644 筆）
✅ 測試集已存：predict_data/long/eco_SYD.csv（共 1986 筆）
✅ 測試集已存：predict_data/long/eco_ZRH.csv（共 1367 筆）
✅ 測試集已存：predict_data/long/biz_CDG.csv（共 1271 筆）
✅ 測試集已存：predict_data/long/biz_FRA.csv（共 1675 筆）
✅ 測試集已存：predict_data/long/biz_JFK.csv（共 1235 筆）
✅ 測試集已存：predict_data/long/biz_LAX.csv（共 2240 筆）
✅ 測試集已存：predict_data/long/biz_LHR.csv（共 2003 筆）
✅ 測試集已存：predict_data/long/biz_SYD.csv（共 1603 筆）
✅ 測試集已存：predict_data/long/biz_ZRH.csv（共 1096 筆）
